In [630]:
from sklearn.experimental import enable_halving_search_cv
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error, root_mean_squared_error
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, HalvingRandomSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from math import sqrt

In [631]:
#  api variables 
lag_days_range = 7
user_prediction_range = 7
user_splitting_method = 'custom'


# def _setup(user_defined_setting):
#     if
    

In [632]:
data = [{'quantity': '5', 'created_at': '2024-01-01', 'positive': 5, 'negative': 0}, {'quantity': '9', 'created_at': '2024-01-02', 'positive': 3, 'negative': 6}, {'quantity': '5', 'created_at': '2024-01-03', 'positive': 3, 'negative': 2}, {'quantity': '5', 'created_at': '2024-01-04', 'positive': 4, 'negative': 1}, {'created_at': '2024-01-05', 'quantity': 0}, {'quantity': '4', 'created_at': '2024-01-06', 'positive': 2, 'negative': 2}, {'quantity': '5', 'created_at': '2024-01-07', 'positive': 1, 'negative': 4}, {'quantity': '4', 'created_at': '2024-01-08', 'positive': 1, 'negative': 3}, {'quantity': '9', 'created_at': '2024-01-09', 'positive': 4, 'negative': 5}, {'quantity': '3', 'created_at': '2024-01-10', 'positive': 3, 'negative': 0}, {'quantity': '1', 'created_at': '2024-01-11', 'positive': 0, 'negative': 1}, {'quantity': '5', 'created_at': '2024-01-12', 'positive': 5, 'negative': 0}, {'quantity': '9', 'created_at': '2024-01-13', 'positive': 9, 'negative': 0}, {'quantity': '6', 'created_at': '2024-01-14', 'positive': 4, 'negative': 2}, {'quantity': '6', 'created_at': '2024-01-15', 'positive': 2, 'negative': 4}, {'quantity': '8', 'created_at': '2024-01-16', 'positive': 8, 'negative': 0}, {'quantity': '5', 'created_at': '2024-01-17', 'positive': 5, 'negative': 0}, {'quantity': '2', 'created_at': '2024-01-18', 'positive': 2, 'negative': 0}, {'quantity': '7', 'created_at': '2024-01-19', 'positive': 5, 'negative': 2}, {'quantity': '5', 'created_at': '2024-01-20', 'positive': 4, 'negative': 1}, {'quantity': '9', 'created_at': '2024-01-21', 'positive': 9, 'negative': 0}, {'quantity': '4', 'created_at': '2024-01-22', 'positive': 2, 'negative': 2}, {'quantity': '6', 'created_at': '2024-01-23', 'positive': 2, 'negative': 4}, {'quantity': '3', 'created_at': '2024-01-24', 'positive': 0, 'negative': 3}, {'quantity': '5', 'created_at': '2024-01-25', 'positive': 4, 'negative': 1}, {'created_at': '2024-01-26', 'quantity': 0}, {'quantity': '6', 'created_at': '2024-01-27', 'positive': 2, 'negative': 4}, {'quantity': '7', 'created_at': '2024-01-28', 'positive': 1, 'negative': 6}, {'quantity': '6', 'created_at': '2024-01-29', 'positive': 4, 'negative': 2}, {'quantity': '5', 'created_at': '2024-01-30', 'positive': 2, 'negative': 3}, {'quantity': '6', 'created_at': '2024-01-31', 'positive': 2, 'negative': 4}, {'quantity': '6', 'created_at': '2024-02-01', 'positive': 0, 'negative': 6}, {'quantity': '5', 'created_at': '2024-02-02', 'positive': 5, 'negative': 0}, {'created_at': '2024-02-03', 'quantity': 0}, {'quantity': '3', 'created_at': '2024-02-04', 'positive': 3, 'negative': 0}, {'quantity': '6', 'created_at': '2024-02-05', 'positive': 6, 'negative': 0}, {'quantity': '7', 'created_at': '2024-02-06', 'positive': 6, 'negative': 1}, {'quantity': '10', 'created_at': '2024-02-07', 'positive': 10, 'negative': 0}, {'quantity': '10', 'created_at': '2024-02-08', 'positive': 3, 'negative': 7}, {'quantity': '7', 'created_at': '2024-02-09', 'positive': 7, 'negative': 0}, {'quantity': '10', 'created_at': '2024-02-10', 'positive': 4, 'negative': 6}, {'quantity': '11', 'created_at': '2024-02-11', 'positive': 11, 'negative': 0}, {'quantity': '7', 'created_at': '2024-02-12', 'positive': 7, 'negative': 0}, {'created_at': '2024-02-13', 'quantity': 0}, {'quantity': '5', 'created_at': '2024-02-14', 'positive': 1, 'negative': 4}, {'quantity': '1', 'created_at': '2024-02-15', 'positive': 1, 'negative': 0}, {'quantity': '7', 'created_at': '2024-02-16', 'positive': 4, 'negative': 3}, {'quantity': '5', 'created_at': '2024-02-17', 'positive': 4, 'negative': 1}, {'quantity': '3', 'created_at': '2024-02-18', 'positive': 0, 'negative': 3}, {'quantity': '5', 'created_at': '2024-02-19', 'positive': 4, 'negative': 1}, {'quantity': '6', 'created_at': '2024-02-20', 'positive': 5, 'negative': 1}, {'quantity': '9', 'created_at': '2024-02-21', 'positive': 7, 'negative': 2}, {'quantity': '6', 'created_at': '2024-02-22', 'positive': 6, 'negative': 0}, {'quantity': '5', 'created_at': '2024-02-23', 'positive': 5, 'negative': 0}, {'quantity': '9', 'created_at': '2024-02-24', 'positive': 6, 'negative': 3}, {'created_at': '2024-02-25', 'quantity': 0}, {'quantity': '5', 'created_at': '2024-02-26', 'positive': 5, 'negative': 0}, {'quantity': '6', 'created_at': '2024-02-27', 'positive': 6, 'negative': 0}, {'quantity': '10', 'created_at': '2024-02-28', 'positive': 10, 'negative': 0}, {'quantity': '6', 'created_at': '2024-02-29', 'positive': 6, 'negative': 0}, {'quantity': '10', 'created_at': '2024-03-01', 'positive': 10, 'negative': 0}, {'quantity': '5', 'created_at': '2024-03-02', 'positive': 4, 'negative': 1}, {'quantity': '8', 'created_at': '2024-03-03', 'positive': 4, 'negative': 4}, {'quantity': '9', 'created_at': '2024-03-04', 'positive': 9, 'negative': 0}, {'quantity': '6', 'created_at': '2024-03-05', 'positive': 1, 'negative': 5}, {'quantity': '6', 'created_at': '2024-03-06', 'positive': 5, 'negative': 1}, {'quantity': '6', 'created_at': '2024-03-07', 'positive': 6, 'negative': 0}, {'quantity': '9', 'created_at': '2024-03-08', 'positive': 9, 'negative': 0}, {'quantity': '1', 'created_at': '2024-03-09', 'positive': 0, 'negative': 1}, {'quantity': '2', 'created_at': '2024-03-10', 'positive': 2, 'negative': 0}, {'quantity': '3', 'created_at': '2024-03-11', 'positive': 2, 'negative': 1}, {'quantity': '6', 'created_at': '2024-03-12', 'positive': 6, 'negative': 0}, {'quantity': '4', 'created_at': '2024-03-13', 'positive': 3, 'negative': 1}, {'quantity': '5', 'created_at': '2024-03-14', 'positive': 4, 'negative': 1}, {'created_at': '2024-03-15', 'quantity': 0}, {'quantity': '5', 'created_at': '2024-03-16', 'positive': 4, 'negative': 1}, {'quantity': '7', 'created_at': '2024-03-17', 'positive': 7, 'negative': 0}, {'quantity': '7', 'created_at': '2024-03-18', 'positive': 6, 'negative': 1}, {'quantity': '1', 'created_at': '2024-03-19', 'positive': 1, 'negative': 0}, {'quantity': '5', 'created_at': '2024-03-20', 'positive': 2, 'negative': 3}, {'quantity': '8', 'created_at': '2024-03-21', 'positive': 1, 'negative': 7}, {'created_at': '2024-03-22', 'quantity': 0}, {'quantity': '11', 'created_at': '2024-03-23', 'positive': 11, 'negative': 0}, {'quantity': '3', 'created_at': '2024-03-24', 'positive': 3, 'negative': 0}, {'quantity': '11', 'created_at': '2024-03-25', 'positive': 5, 'negative': 6}, {'quantity': '1', 'created_at': '2024-03-26', 'positive': 0, 'negative': 1}, {'created_at': '2024-03-27', 'quantity': 0}, {'quantity': '7', 'created_at': '2024-03-28', 'positive': 7, 'negative': 0}, {'quantity': '5', 'created_at': '2024-03-29', 'positive': 4, 'negative': 1}, {'quantity': '6', 'created_at': '2024-03-30', 'positive': 6, 'negative': 0}, {'quantity': '1', 'created_at': '2024-03-31', 'positive': 0, 'negative': 1}, {'quantity': '6', 'created_at': '2024-04-01', 'positive': 6, 'negative': 0}, {'quantity': '3', 'created_at': '2024-04-02', 'positive': 0, 'negative': 3}, {'quantity': '8', 'created_at': '2024-04-03', 'positive': 8, 'negative': 0}, {'quantity': '3', 'created_at': '2024-04-04', 'positive': 2, 'negative': 1}, {'quantity': '5', 'created_at': '2024-04-05', 'positive': 5, 'negative': 0}, {'quantity': '8', 'created_at': '2024-04-06', 'positive': 4, 'negative': 4}, {'quantity': '2', 'created_at': '2024-04-07', 'positive': 2, 'negative': 0}, {'created_at': '2024-04-08', 'quantity': 0}, {'quantity': '8', 'created_at': '2024-04-09', 'positive': 6, 'negative': 2}, {'quantity': '6', 'created_at': '2024-04-10', 'positive': 6, 'negative': 0}, {'quantity': '6', 'created_at': '2024-04-11', 'positive': 5, 'negative': 1}, {'quantity': '9', 'created_at': '2024-04-12', 'positive': 5, 'negative': 4}, {'quantity': '11', 'created_at': '2024-04-13', 'positive': 9, 'negative': 2}, {'quantity': '4', 'created_at': '2024-04-14', 'positive': 4, 'negative': 0}, {'quantity': '10', 'created_at': '2024-04-15', 'positive': 4, 'negative': 6}, {'quantity': '8', 'created_at': '2024-04-16', 'positive': 8, 'negative': 0}, {'quantity': '7', 'created_at': '2024-04-17', 'positive': 7, 'negative': 0}, {'quantity': '11', 'created_at': '2024-04-18', 'positive': 2, 'negative': 9}, {'quantity': '10', 'created_at': '2024-04-19', 'positive': 3, 'negative': 7}, {'quantity': '7', 'created_at': '2024-04-20', 'positive': 6, 'negative': 1}, {'quantity': '8', 'created_at': '2024-04-21', 'positive': 8, 'negative': 0}, {'quantity': '11', 'created_at': '2024-04-22', 'positive': 10, 'negative': 1}, {'quantity': '9', 'created_at': '2024-04-23', 'positive': 8, 'negative': 1}, {'quantity': '7', 'created_at': '2024-04-24', 'positive': 7, 'negative': 0}, {'quantity': '11', 'created_at': '2024-04-25', 'positive': 6, 'negative': 5}, {'quantity': '7', 'created_at': '2024-04-26', 'positive': 3, 'negative': 4}, {'quantity': '5', 'created_at': '2024-04-27', 'positive': 5, 'negative': 0}, {'quantity': '7', 'created_at': '2024-04-28', 'positive': 7, 'negative': 0}, {'quantity': '6', 'created_at': '2024-04-29', 'positive': 0, 'negative': 6}, {'quantity': '6', 'created_at': '2024-04-30', 'positive': 6, 'negative': 0}]

In [633]:
# FEATURES_1 = ['month', 'day_of_month', 'day_of_year', 'week_of_year', 'day_of_week','year','is_wknd','is_month_start','is_month_end']
# TARGET = 'quantity'

In [634]:
# check if data is a list and convert to pandas dataframe
if data and isinstance(data, list):
    print('Data is a list')
    try:
        # convert data to a pandas dataframe
        df = pd.DataFrame(data)
        # convert created_at to datetime value
        df['created_at'] = pd.to_datetime(df['created_at'])
        # set created_at as index
        df.index = df['created_at']
        # drop created_at column because it is now the index
        df.drop('created_at', axis=1, inplace=True)
        print('Dataframe created successfully')
    except KeyError:
        print('KeyError: created_at column not found')
else:
    print('data is not a list')
    

Data is a list
Dataframe created successfully


In [635]:
# df.fillna(np.nan, inplace=True)
df['quantity'] = df['quantity'].astype(float)
df['positive'] = df['positive'].astype(float)
# print(df.dtypes)
# df.head()

In [636]:
df.dropna(inplace=True)
df.isnull().sum()

quantity    0
positive    0
negative    0
dtype: int64

In [637]:
df.shape

(112, 3)

In [639]:
from datetime import timedelta

# take only a 7 days of the df data
df = df[df.index < df.index.min() + timedelta(days=7)]

In [640]:
min = df.index.min()
max = df.index.max()
print(f'min: {min}, max: {max}')

min: 2024-01-01 00:00:00, max: 2024-01-07 00:00:00


In [641]:
df

,quantity,positive,negative
created_at,,,
2024-01-01,5.0,5.0,0.0
2024-01-02,9.0,3.0,6.0
2024-01-03,5.0,3.0,2.0
2024-01-04,5.0,4.0,1.0
2024-01-06,4.0,2.0,2.0
2024-01-07,5.0,1.0,4.0


# FEATURE ENGINEERING

In [642]:
def generate_date_features(dataframe: pd.DataFrame):
    df_copy = dataframe.copy()  # Create a copy of the DataFrame to ensure the original remains unchanged

    df_copy['month'] = df_copy.index.month    
    df_copy['day_of_month'] = df_copy.index.day
    df_copy['day_of_year'] = df_copy.index.dayofyear
    df_copy['week_of_year'] = df_copy.index.isocalendar().week
    df_copy['day_of_week'] = df_copy.index.dayofweek
    df_copy['year'] = df_copy.index.year
    df_copy["is_wknd"] = df_copy.index.weekday // 4
    df_copy['is_month_start'] = df_copy.index.is_month_start.astype(int)
    df_copy['is_month_end'] = df_copy.index.is_month_end.astype(int)
    
    return df_copy

target_map = df['quantity'].to_dict()

def generate_lag_features(dataframe: pd.DataFrame):
    df_copy = dataframe.copy()
    # create lag features
    for i in range(lag_days_range + 1):
        df_copy[f'quantity_lag_{i}'] = df_copy['quantity'].shift(i)
        df_copy[f'positive_lag_{i}'] = df_copy['positive'].shift(i)
        df_copy[f'negative_lag_{i}'] = df_copy['negative'].shift(i)
        # # convert to int
        
    # df_copy['lag_1'] = (df.index - pd.Timedelta(days=30)).map(target_map)
    # df_copy['lag_2'] = (df.index - pd.Timedelta(days=60)).map(target_map)
    # df_copy['lag_3'] = (df.index - pd.Timedelta(days=90)).map(target_map)
    return df_copy

# APPLY FEATURE ENGINEERING

In [643]:
df_w_date_features = generate_date_features(df)
df_w_date_lag_features = generate_lag_features(df_w_date_features)

# DROP MISSING VALUES

In [644]:
print(f'df zero value count: {df_w_date_lag_features.isnull().sum()}')
# count all 0 value in quantity column
print(f'quantity zero value count: {df_w_date_lag_features[df_w_date_lag_features["quantity"] == 0].shape[0]}')

df zero value count: quantity          0
positive          0
negative          0
month             0
day_of_month      0
day_of_year       0
week_of_year      0
day_of_week       0
year              0
is_wknd           0
is_month_start    0
is_month_end      0
quantity_lag_0    0
positive_lag_0    0
negative_lag_0    0
quantity_lag_1    1
positive_lag_1    1
negative_lag_1    1
quantity_lag_2    2
positive_lag_2    2
negative_lag_2    2
quantity_lag_3    3
positive_lag_3    3
negative_lag_3    3
quantity_lag_4    4
positive_lag_4    4
negative_lag_4    4
quantity_lag_5    5
positive_lag_5    5
negative_lag_5    5
quantity_lag_6    6
positive_lag_6    6
negative_lag_6    6
quantity_lag_7    6
positive_lag_7    6
negative_lag_7    6
dtype: int64
quantity zero value count: 0


In [645]:
df_w_date_lag_features

,quantity,positive,negative,month,day_of_month,day_of_year,week_of_year,day_of_week,year,is_wknd,...,negative_lag_4,quantity_lag_5,positive_lag_5,negative_lag_5,quantity_lag_6,positive_lag_6,negative_lag_6,quantity_lag_7,positive_lag_7,negative_lag_7
created_at,,,,,,,,,,,,,,,,,,,,,
2024-01-01,5.0,5.0,0.0,1,1,1,1,0,2024,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-02,9.0,3.0,6.0,1,2,2,1,1,2024,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-03,5.0,3.0,2.0,1,3,3,1,2,2024,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-04,5.0,4.0,1.0,1,4,4,1,3,2024,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-06,4.0,2.0,2.0,1,6,6,1,5,2024,1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-07,5.0,1.0,4.0,1,7,7,1,6,2024,1,...,6.0,5.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [646]:
# df_w_features.dtypes

# GENERATE LAG FEATURES

In [647]:
# user_defined_lag = 7
# user_defined_days = 30 # get from the api
# 
# def generate_lag_features(df):
#     df_copy = df.copy()
#     # create lag features
#     for i in range(user_defined_days + 1):
#         df_copy[f'quantity_lag_{i}'] = df_copy['quantity'].shift(i)
#         df_copy[f'positive_lag_{i}'] = df_copy['positive'].shift(i)
#         df_copy[f'negative_lag_{i}'] = df_copy['negative'].shift(i)
#         # # convert to int
#         
#     # df_copy['lag_1'] = (df.index - pd.Timedelta(days=7)).map(target_map)
#     # df_copy['lag_2'] = (df.index - pd.Timedelta(days=14)).map(target_map)
#     # df_copy['lag_3'] = (df.index - pd.Timedelta(days=21)).map(target_map)
#     return df_copy
#     
# # df_with_lag_features = generate_lag_features(df_w_features)

In [648]:
# print(f'df zero value count: {df_with_lag_features.isnull().sum()}')
# df_with_lag_features.fillna(np.nan, inplace=True)

#convert to int
# df_with_lag_features = df_with_lag_features.astype(int)

# df_with_lag_features.head(10)

# df_with_lag_features

# TIME SERIES CROSS VALIDATION

from sklearn.model_selection import TimeSeriesSplit
import numpy as np

# Example data (7 days worth of data)
data = np.random.rand(7)  # Replace this with your actual data

# Number of splits (e.g., 5-fold cross-validation)
n_splits = 5

# Create TimeSeriesSplit object
tscv = TimeSeriesSplit(n_splits=n_splits)

# Iterate over the splits and print the train and test indices
for train_index, test_index in tscv.split(data):
    print("Train indices:", train_index)
    print("Test indices:", test_index)
    print("---")


# TRAIN TEST SPLIT

In [649]:
import pandas as pd

def time_based_split(data, split_ratio=0.8):
    """
    Splits a time series dataset into training and testing sets based on a time-based split ratio.

    Args:
        data (pd.Series or pd.DataFrame): The time series data to split.
        split_ratio (float): The proportion of data to use for training (0 to 1). Default is 0.8.

    Returns:
        tuple: A tuple containing the training and testing sets as pandas DataFrames.
    """

    total_length = len(data)
    train_size = int(total_length * split_ratio)
    test_size = total_length - train_size

    train = data.iloc[:train_size]
    test = data.iloc[train_size:]

    return train, test


In [650]:
# # train test split
# train, test = time_based_split(df, split_ratio=0.8)
# 
# print(f'train shape: {train.shape}, test shape: {test.shape}')
# print(f'train min and max: {train.index.min()}, {train.index.max()}')
# print(f'test min and max: {test.index.min()}, {test.index.max()}')

In [651]:
def handle_train_test_size(df, splitting_method, split_ratio, custom_days=None):
    if splitting_method == 'time_based_split':
        if split_ratio > 1 or split_ratio < 0:
            raise ValueError('split_ratio must be between 0 and 1')
        
        train, test = time_based_split(df, split_ratio)
        
        print(f'train shape: {train.shape}, test shape: {test.shape}')
        print(f'train min and max: {train.index.min()}, {train.index.max()}')
        print(f'test min and max: {test.index.min()}, {test.index.max()}')
        
        return train, test
    
    elif splitting_method == 'week':
        end_data = df.index.max()
        # subtract 7 days from end_date
        start_data = end_data - pd.Timedelta(days=7)
        
        train = df[df.index < start_data]
        test = df[df.index >= start_data]
        
        print(f'train shape: {train.shape}, test shape: {test.shape}')
        print(f'train min and max: {train.index.min()}, {train.index.max()}')
        print(f'test min and max: {test.index.min()}, {test.index.max()}')
        
        return train, test
    
    elif splitting_method == 'month':
        end_data = df.index.max()
        # subtract 30 days from end_date
        start_data = end_data - pd.Timedelta(days=30)
        
        train = df[df.index < start_data]
        test = df[df.index >= start_data]
        
        print(f'train shape: {train.shape}, test shape: {test.shape}')
        print(f'train min and max: {train.index.min()}, {train.index.max()}')
        print(f'test min and max: {test.index.min()}, {test.index.max()}')
        
        return train, test
    
    elif splitting_method == 'custom':
        end_data = df.index.max()
        # subtract 30 days from end_date
        start_data = end_data - pd.Timedelta(days=custom_days)
        
        train = df[df.index < start_data]
        test = df[df.index >= start_data]
        
        print(f'train shape: {train.shape}, test shape: {test.shape}')
        print(f'train min and max: {train.index.min()}, {train.index.max()}')
        print(f'test min and max: {test.index.min()}, {test.index.max()}')
        
        return train, test
    else:
        raise ValueError('splitting_method must be either time_based_split, week or month, custom')

# TRAIN TEST SPLIT

In [653]:
train, test = handle_train_test_size(df_w_date_lag_features, splitting_method='custom', custom_days=2, split_ratio=0.8)

train shape: (4, 36), test shape: (2, 36)
train min and max: 2024-01-01 00:00:00, 2024-01-04 00:00:00
test min and max: 2024-01-06 00:00:00, 2024-01-07 00:00:00


In [654]:
train

,quantity,positive,negative,month,day_of_month,day_of_year,week_of_year,day_of_week,year,is_wknd,...,negative_lag_4,quantity_lag_5,positive_lag_5,negative_lag_5,quantity_lag_6,positive_lag_6,negative_lag_6,quantity_lag_7,positive_lag_7,negative_lag_7
created_at,,,,,,,,,,,,,,,,,,,,,
2024-01-01,5.0,5.0,0.0,1,1,1,1,0,2024,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-02,9.0,3.0,6.0,1,2,2,1,1,2024,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-03,5.0,3.0,2.0,1,3,3,1,2,2024,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-04,5.0,4.0,1.0,1,4,4,1,3,2024,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [655]:
# train, test = handle_train_test_size(df_w_date_lag_features, 'time_based_split', 0.8, custom_days=7)

# TRAIN TEST SPLIT WEEK METHOD

In [656]:
# train, test = handle_train_test_size(df_w_date_lag_features, 'week', 0.8, custom_days=7)

# TRAIN TEST SPLIT MONTH METHOD

In [657]:
# train, test = handle_train_test_size(df_w_date_lag_features, 'month', 0.8, custom_days=7)

# TRAIN TEST SPLIT CUSTOM METHOD

In [658]:
# train, test = handle_train_test_size(df_w_date_lag_features, 'custom', 0.8, custom_days=10)

In [659]:
# # train test split
# 
# 
# train = df_with_lag_features[df_with_lag_features.index < '2024-03-01']
# test = df_with_lag_features[df_with_lag_features.index >= '2024-03-01']
# 
# 
# 
# print(f'train shape: {train.shape}, test shape: {test.shape}')
# print(f'train min and max: {train.index.min()}, {train.index.max()}')
# print(f'test min and max: {test.index.min()}, {test.index.max()}')

In [660]:
def create_features_target(dataframe, target):
    DATE_FEATURES = ['month', 'day_of_month', 'day_of_year', 'week_of_year', 'day_of_week','year','is_wknd','is_month_start','is_month_end']
    LAG_FEATURES = [col for col in dataframe.columns if 'quantity_lag' in col or 'positive_lag' in col or 'negative_lag' in col or 'lag' in col]
    FEATURES = DATE_FEATURES + LAG_FEATURES
    TARGET = target
    return FEATURES, TARGET

In [661]:
FEATURES, TARGET = create_features_target(train, 'quantity')

In [662]:
TARGET

'quantity'

In [663]:
FEATURES

['month',
 'day_of_month',
 'day_of_year',
 'week_of_year',
 'day_of_week',
 'year',
 'is_wknd',
 'is_month_start',
 'is_month_end',
 'quantity_lag_0',
 'positive_lag_0',
 'negative_lag_0',
 'quantity_lag_1',
 'positive_lag_1',
 'negative_lag_1',
 'quantity_lag_2',
 'positive_lag_2',
 'negative_lag_2',
 'quantity_lag_3',
 'positive_lag_3',
 'negative_lag_3',
 'quantity_lag_4',
 'positive_lag_4',
 'negative_lag_4',
 'quantity_lag_5',
 'positive_lag_5',
 'negative_lag_5',
 'quantity_lag_6',
 'positive_lag_6',
 'negative_lag_6',
 'quantity_lag_7',
 'positive_lag_7',
 'negative_lag_7']

In [664]:
def create_train_data(dataframe, features, target):
    X_train = dataframe[features]
    y_train = dataframe[target]
    return X_train, y_train

def create_test_data(dataframe, features, target):
    X_test = dataframe[features]
    y_test = dataframe[target]
    return X_test, y_test

In [665]:
X_train, y_train = create_train_data(train, FEATURES, TARGET)
X_test, y_test = create_test_data(test, FEATURES, TARGET)

In [666]:
X_train.shape, y_train.shape

((4, 33), (4,))

In [667]:
X_test.shape, y_test.shape

((2, 33), (2,))

# MODEL TRAINING

In [668]:
from random import randint, uniform
from scipy.stats import loguniform

# Define the parameter grid
param_grid = {
    'n_estimators': [500, 1000, 2000],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3],
    'subsample': [0.6, 0.8, 1.0],
    'min_child_weight': [1, 3, 5]
}
param_dist = {
    'n_estimators': [500, 1000, 2000],  # List of specific values for n_estimators
    'max_depth': [3, 5, 7],
    'learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3],
    'subsample': [0.6, 0.8, 1.0],
    'min_child_weight': [1, 3, 5]
}


# Initialize the XGBRegressor
reg = xgb.XGBRegressor(
    base_score=0.5,
    booster='gbtree',
    objective='reg:squarederror',
    # objective='reg:squaredlogerror',
    early_stopping_rounds=50,
    eval_metric='mae',
    n_jobs=-1,
)

tscv = TimeSeriesSplit(n_splits=2)

# Initialize the GridSearchCV
grid_search = GridSearchCV(estimator=reg, param_grid=param_grid, cv=tscv, n_jobs=-1)
# grid_search = HalvingRandomSearchCV(estimator=reg, param_distributions=param_dist, cv=tscv, n_jobs=-1)

# Fit the GridSearchCV to the data
grid_search.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], verbose=0)

# Get the best parameters
best_params = grid_search.best_params_

# Print the best parameters
print(f'Best parameters: {best_params}')

/run/media/magi/8ae8cf59-85ae-4153-b477-636270c3addc/ext-magi/Programming/CAPSTONE_FRONTEND/capstone-prtech/Python/stock-sense-api/.venv/lib/python3.11/site-packages/sklearn/metrics/_regression.py:1187: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/run/media/magi/8ae8cf59-85ae-4153-b477-636270c3addc/ext-magi/Programming/CAPSTONE_FRONTEND/capstone-prtech/Python/stock-sense-api/.venv/lib/python3.11/site-packages/sklearn/metrics/_regression.py:1187: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/run/media/magi/8ae8cf59-85ae-4153-b477-636270c3addc/ext-magi/Programming/CAPSTONE_FRONTEND/capstone-prtech/Python/stock-sense-api/.venv/lib/python3.11/site-packages/sklearn/metrics/_regression.py:1187: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/run/media

Best parameters: {'learning_rate': 0.0001, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 500, 'subsample': 0.6}


In [669]:
# # Train the model using the best parameters
# stopping_round = {'early_stopping_rounds': 50}
# 
# best_params = {
#     **best_params,?
#     **stopping_round
# }

print(f'Best parameters: {best_params}')

reg = xgb.XGBRegressor(
    base_score=0.5,
    booster='gbtree',
    objective='reg:squarederror',
    # objective='reg:squaredlogerror',
    early_stopping_rounds=50,
    eval_metric='mae',
    n_jobs=-1,
    **best_params
)
reg.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=1000
)

Best parameters: {'learning_rate': 0.0001, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 500, 'subsample': 0.6}
[0]	validation_0-mae:5.49956	validation_1-mae:3.99956
[499]	validation_0-mae:5.28520	validation_1-mae:3.78520


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=50,
             enable_categorical=False, eval_metric='mae', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.0001, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=-1,
             num_parallel_tree=None, random_state=None, ...)

In [670]:
def train_model(best_params, X_train, y_train, X_test=None, y_test=None):
    """
    Trains a model using the best parameters found by GridSearchCV.
    :param best_params: The best parameters found by GridSearchCV.
    :param X_train: The training features.
    :param y_train: The training target.
    :param X_test: The testing features.
    :param y_test: The testing target.
    :return: The trained model.
    """
    try:
        print('Training model...')

        # Initialize the XGBRegressor
        reg = xgb.XGBRegressor(
            base_score=0.5,
            booster='gbtree',
            objective='reg:squarederror',
            # objective='reg:squaredlogerror',
            early_stopping_rounds=50,
            eval_metric='mae',
            n_jobs=-1,
            **best_params
        )
        
        if X_test is not None and y_test is not None:
            print("X_test and y_test are not None, Running model with test data for evaluation")
            
            reg.fit(
                X_train, y_train,
                eval_set=[(X_train, y_train), (X_test, y_test)],
                verbose=1000
            )
        else:
            print("Either X_test or y_test or both are None, Running model without test data for future prediction")
            
            reg.fit(
                X_train, y_train,
                eval_set=[(X_train, y_train)],
                verbose=1000
            )
            
        print('Model trained successfully')
        return reg

    except Exception as e:
        print(f"An error occurred: {e}")
        return False

In [671]:
# print(type(X_test))

In [672]:
reg = train_model(best_params, X_train, y_train, X_test, y_test)

Training model...
X_test and y_test are not None, Running model with test data for evaluation
[0]	validation_0-mae:5.49956	validation_1-mae:3.99956
[499]	validation_0-mae:5.28520	validation_1-mae:3.78520
Model trained successfully


In [673]:
def generate_Xtrain_ytrain(dataframe, features, target):
    X_train = dataframe[features]
    y_train = dataframe[target]
    return X_train, y_train

# MODEL EVALUATION

In [674]:
# print(FEATURES_COMBINED)

In [675]:
# Generate predictions
y_pred_train = reg.predict(X_train)
y_pred_test = reg.predict(X_test)

# loop to y_preds_test and round them up to nearast integer
# y_pred_test = [round(i) for i in y_pred_test]

# Calculate metrics for the training set
mse_train = mean_squared_error(y_train, y_pred_train)
mae_train = mean_absolute_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)
mape_train = mean_absolute_percentage_error(y_train, y_pred_train)
rmse_train = sqrt(mse_train)

# Calculate metrics for the test set
mse_test = mean_squared_error(y_test, y_pred_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)
mape_test = mean_absolute_percentage_error(y_test, y_pred_test)
rmse_test = root_mean_squared_error(y_test, y_pred_test)
nrmse = rmse_test / (y_test.max() - y_test.min())  # Calculate NRMSE

# Print the metrics
print(f'Training set metrics: MSE={mse_train}, MAE={mae_train}, R2={r2_train}, MAPE={mape_train}, RMSE={rmse_train}')
print(f'Test set metrics: MSE={mse_test}, MAE={mae_test}, R2={r2_test}, MAPE={mape_test}, RMSE={rmse_test}, NRMSE={nrmse}')
y_pred_test

Training set metrics: MSE=30.93329619494456, MAE=5.28519594669342, R2=-9.311098731648187, MAPE=0.8729237238566081, RMSE=5.561770958511737
Test set metrics: MSE=14.5777083548643, MAE=3.7851959466934204, R2=-57.3108334194572, MAPE=0.8391690880060196, RMSE=3.8180765255379967, NRMSE=3.8180765255379967


array([0.71480405, 0.71480405], dtype=float32)

In [676]:
def mode_evaluate(model, X_test, y_test):
    try:
        print('Evaluating model')
        y_pred_test = model.predict(X_test)
        

        # Calculate metrics for the test set
        mse_test = mean_squared_error(y_test, y_pred_test)
        mae_test = mean_absolute_error(y_test, y_pred_test)
        r2_test = r2_score(y_test, y_pred_test)
        mape_test = mean_absolute_percentage_error(y_test, y_pred_test)
        rmse_test = root_mean_squared_error(y_test, y_pred_test)
        nrmse = rmse_test / (y_test.max() - y_test.min())  # Calculate NRMSE
        
        # create a dataframe with index of X_test  and columns of y_pred_test and y_test 
        test_df = pd.DataFrame({'date': X_test.index ,'actual': y_test, 'predicted': y_pred_test}, index=X_test.index)
        
        test_df['date'] = test_df['date'].dt.strftime('%Y-%m-%d')
    
        print(
            f'Test set metrics: MSE={mse_test}, MAE={mae_test}, R2={r2_test}, MAPE={mape_test}, RMSE={rmse_test}, NRMSE={nrmse}')

        accuracy = {
            'mse': mse_test,
            'mae': mae_test,
            'r2': r2_test,
            'mape': mape_test,
            'rmse': rmse_test,
            'nrmse': nrmse,
            'acurracy_test': test_df.to_dict('records'),
        }
        print('Model evaluated successfully')
        return accuracy
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return False

In [677]:
accuracy = mode_evaluate(reg, X_test, y_test)

Evaluating model
Test set metrics: MSE=14.5777083548643, MAE=3.7851959466934204, R2=-57.3108334194572, MAPE=0.8391690880060196, RMSE=3.8180765255379967, NRMSE=3.8180765255379967
Model evaluated successfully


In [678]:
accuracy

{'mse': 14.5777083548643,
 'mae': 3.7851959466934204,
 'r2': -57.3108334194572,
 'mape': 0.8391690880060196,
 'rmse': 3.8180765255379967,
 'nrmse': 3.8180765255379967,
 'acurracy_test': [{'date': '2024-01-06',
   'actual': 4.0,
   'predicted': 0.7148040533065796},
  {'date': '2024-01-07', 'actual': 5.0, 'predicted': 0.7148040533065796}]}

In [679]:
y_pred_test

array([0.71480405, 0.71480405], dtype=float32)

In [680]:
# test_w_prediction = test.copy()
# test_w_prediction['quantity_pred'] = reg.predict(X_test)
# 
# # Display the first few rows of the test_w_prediction DataFrame
# test_w_prediction.head()

In [681]:
# plot quanity and quantity_pred
# plt.figure(figsize=(15, 7))
# plt.plot(test_w_prediction.index, test_w_prediction['quantity'], label='quantity')
# plt.plot(test_w_prediction.index, test_w_prediction['quantity_pred'], label='quantity_pred')
# plt.legend()
# plt.show()

# MODEL EVALUATION

# GENERATE FUTURE DATES

In [682]:
# max = df.index.max()
# add 1 day to max because we want to predict for the next 7 days, and we have today's sales data
# max = max + pd.Timedelta(days=1)

In [683]:
# predictfornext = pd.date_range(max, periods=7, freq='D')

In [684]:
# predictfornext

In [685]:
def generate_future_target_dates(last_date_of_data, pred_range: int):
    # Add 1 day to max because we want to predict for the next 7 days, and we have today's sales data
    end_date = last_date_of_data + pd.Timedelta(days=1)
    # create a Datetimeindex from pass variables
    future_dates = pd.date_range(end_date, periods=pred_range, freq='D')
    # Create a single column called date
    future_dates_df = pd.DataFrame(future_dates, columns=['date'])
    # Make the date the first column the index
    future_dates_df.set_index('date', inplace=True)
    # Add quantity column
    future_dates_df['quantity'] = np.nan
    # add isFuture column
    future_dates_df['isFuture'] = True
    # # create date features
    # future_dates_df = create_date_features(future_dates_df)
    return future_dates_df


def generate_future_w_lags_dates(orignal_dataframe, future_dataframe):
    try:
        original_dataframe_copy = orignal_dataframe.copy()
        
        original_dataframe_copy['isFuture'] = False
        
        # get the original dataframe and concat with the future dates dataframe
        future_dates_df = pd.concat([original_dataframe_copy, future_dataframe])
        
        future_dates_df = generate_date_features(future_dates_df)
        future_dates_df = generate_lag_features(future_dates_df)
        
        return future_dates_df
    except Exception as e:
        print(f"An error occurred: {e}")
        return False
    
    
def generate_prediction_for_future_dates(model, future_dates_df, features):

    predict_data = future_dates_df.query('isFuture == True').copy()
    
    predict_data['pred'] = model.predict(predict_data[features])
    
    prediction = predict_data[['pred']]
    
    return prediction

In [686]:
max

Timestamp('2024-01-07 00:00:00')

In [687]:
test_df = generate_future_target_dates(max, pred_range=1)

In [688]:
test_df.shape

(1, 2)

In [689]:
# predict_data = test_df.query('isFuture == False').copy()

In [690]:
# predict_data

In [691]:
# test_df

In [692]:
df

,quantity,positive,negative
created_at,,,
2024-01-01,5.0,5.0,0.0
2024-01-02,9.0,3.0,6.0
2024-01-03,5.0,3.0,2.0
2024-01-04,5.0,4.0,1.0
2024-01-06,4.0,2.0,2.0
2024-01-07,5.0,1.0,4.0


In [693]:
test = generate_future_w_lags_dates(df, test_df)

In [694]:
test

,quantity,positive,negative,isFuture,month,day_of_month,day_of_year,week_of_year,day_of_week,year,...,negative_lag_4,quantity_lag_5,positive_lag_5,negative_lag_5,quantity_lag_6,positive_lag_6,negative_lag_6,quantity_lag_7,positive_lag_7,negative_lag_7
2024-01-01,5.0,5.0,0.0,False,1,1,1,1,0,2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-02,9.0,3.0,6.0,False,1,2,2,1,1,2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-03,5.0,3.0,2.0,False,1,3,3,1,2,2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-04,5.0,4.0,1.0,False,1,4,4,1,3,2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-06,4.0,2.0,2.0,False,1,6,6,1,5,2024,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-07,5.0,1.0,4.0,False,1,7,7,1,6,2024,...,6.0,5.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-08,NaN,NaN,NaN,True,1,8,8,2,0,2024,...,2.0,9.0,3.0,6.0,5.0,5.0,0.0,NaN,NaN,NaN


In [695]:
def generate_train_for_forecast(dataframe, features, target):
    complete_data = dataframe.query('isFuture == False').copy()
    FX_train = complete_data[features]
    fy_train = complete_data[target]
    return FX_train, fy_train

In [696]:
FX_train, fy_train = generate_train_for_forecast(test, FEATURES, TARGET)

In [697]:
FX_train.shape, fy_train.shape

((6, 33), (6,))

In [698]:
future_model:xgb.XGBRegressor = train_model(best_params, FX_train, fy_train)

Training model...
Either X_test or y_test or both are None, Running model without test data for future prediction
[0]	validation_0-mae:4.99958
[499]	validation_0-mae:4.78965
Model trained successfully


In [699]:
FX_train

,month,day_of_month,day_of_year,week_of_year,day_of_week,year,is_wknd,is_month_start,is_month_end,quantity_lag_0,...,negative_lag_4,quantity_lag_5,positive_lag_5,negative_lag_5,quantity_lag_6,positive_lag_6,negative_lag_6,quantity_lag_7,positive_lag_7,negative_lag_7
2024-01-01,1,1,1,1,0,2024,0,1,0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-02,1,2,2,1,1,2024,0,0,0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-03,1,3,3,1,2,2024,0,0,0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-04,1,4,4,1,3,2024,0,0,0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-06,1,6,6,1,5,2024,1,0,0,4.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-07,1,7,7,1,6,2024,1,0,0,5.0,...,6.0,5.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [700]:
predict_data

,quantity,isFuture
date,,


In [701]:
test

,quantity,positive,negative,isFuture,month,day_of_month,day_of_year,week_of_year,day_of_week,year,...,negative_lag_4,quantity_lag_5,positive_lag_5,negative_lag_5,quantity_lag_6,positive_lag_6,negative_lag_6,quantity_lag_7,positive_lag_7,negative_lag_7
2024-01-01,5.0,5.0,0.0,False,1,1,1,1,0,2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-02,9.0,3.0,6.0,False,1,2,2,1,1,2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-03,5.0,3.0,2.0,False,1,3,3,1,2,2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-04,5.0,4.0,1.0,False,1,4,4,1,3,2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-06,4.0,2.0,2.0,False,1,6,6,1,5,2024,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-07,5.0,1.0,4.0,False,1,7,7,1,6,2024,...,6.0,5.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-08,NaN,NaN,NaN,True,1,8,8,2,0,2024,...,2.0,9.0,3.0,6.0,5.0,5.0,0.0,NaN,NaN,NaN


In [702]:
# test

In [703]:
output = generate_prediction_for_future_dates(future_model, test, features=FEATURES)

# create a dataframe with index of X_test  and columns of y_pred_test and y_test 
output = pd.DataFrame({'date': output.index , 'predicted': output['pred']}, index=output.index)
        
output['date'] = output['date'].dt.strftime('%Y-%m-%d')

# output = output.set_index('date')

output = output[['date', 'predicted']]

In [704]:
output.to_dict('records')

[{'date': '2024-01-08', 'predicted': 0.7103538513183594}]

In [705]:
output.to_dict('records')

[{'date': '2024-01-08', 'predicted': 0.7103538513183594}]

In [165]:
# output = generate_prediction_for_future_dates(reg, test, TARGET, FEATURES)

In [114]:
# output

In [33]:
# future_dates_df_with_lag_features_copy = future_dates_df_with_lag_features.query('isFuture == True').copy()

In [144]:
# predict = test.query('isFuture == True').copy()
# predict

In [35]:
# future_dates_df_with_lag_features_copy['quantity_pred'] = reg.predict(future_dates_df_with_lag_features_copy[FEATURES_COMBINED])

In [143]:
# future_dates_df_with_lag_features_copy

In [142]:
# reg